
# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import pandas as pd
from sqlalchemy import create_engine

from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("DisasterResponse", engine)
X = df["message"]
Y = df.drop(["id", "message", "original", "genre"], axis=1)
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [23]:
def tokenize(text):
    # Convert to lower case
    text = text.lower()
    
    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [word for word in tokens if word not in stopwords.words("english")]
    
    # Lemmatize
    tokens = [WordNetLemmatizer().lemmatize(word).strip() for word in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [4]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

# Train pipeline
pipeline.fit(X_train, Y_train)

In [15]:
X_test.iloc[5]

'The medical help involves the setting up of U.S. military mobile surgical hospitals, he said.'

In [16]:
Y_test.iloc[5]

related                   1
request                   0
offer                     1
aid_related               1
medical_help              1
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
predict_y = pipeline.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(Y_test[category], predict_y[:, i]))

related
             precision    recall  f1-score   support

          0       0.61      0.44      0.51      1563
          1       0.83      0.91      0.87      4944
          2       0.25      0.36      0.29        47

avg / total       0.78      0.79      0.78      6554

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.93      5443
          1       0.79      0.44      0.57      1111

avg / total       0.88      0.89      0.87      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3884
          1       0.74      0.59      0.66      2670

avg / total       0.75      0.75      0.74      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f8689665620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [48]:
parameters = {
    "clf__estimator__n_estimators": [50, 100, 200],
    "clf__estimator__min_samples_split": [2, 3]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [9]:
# Train model
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100, 200], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
cv_predict_y = pickle_model.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(Y_test[category], cv_predict_y[:, i]))

related
             precision    recall  f1-score   support

          0       0.70      0.40      0.51      1563
          1       0.83      0.94      0.88      4944
          2       0.39      0.38      0.39        47

avg / total       0.80      0.81      0.79      6554

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5443
          1       0.84      0.49      0.61      1111

avg / total       0.89      0.90      0.89      6554

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.80      0.83      0.82      3884
          1       0.74      0.70      0.72      2670

avg / total       0.78      0.78      0.78      6554

medical_help
             precision    recall  f1-sco

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6136
          1       0.86      0.40      0.54       418

avg / total       0.95      0.96      0.95      6554

food
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      5809
          1       0.84      0.60      0.70       745

avg / total       0.94      0.94      0.94      6554

shelter
             precision    recall  f1-score   support

          0       0.94      0.99      0.97      5973
          1       0.83      0.35      0.49       581

avg / total       0.93      0.94      0.92      6554

clothing
             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6456
          1       0.86      0.06      0.11        98

avg / total       0.98      0.99      0.98      6554

money
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      6421
     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [40]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

pipeline2

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [42]:
pipeline2.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [44]:
ada_predict_y = pipeline2.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(Y_test[category], ada_predict_y[:, i]))

related
             precision    recall  f1-score   support

          0       0.69      0.10      0.17      1563
          1       0.77      0.99      0.86      4944
          2       0.33      0.09      0.14        47

avg / total       0.75      0.77      0.69      6554

request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5443
          1       0.77      0.48      0.59      1111

avg / total       0.88      0.89      0.88      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6521
          1       0.12      0.03      0.05        33

avg / total       0.99      0.99      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3884
          1       0.75      0.61      0.67      2670

avg / total       0.76      0.76      0.75      6554

medical_help
             precision    recall  f1-sco

### 9. Export your model as a pickle file

In [46]:
pkl_filename = 'ada_clf_model.pkl'
with open(pkl_filename, 'wb') as file:  
    pickle.dump(pipeline2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.